# Install necessary libs, if the computer don't have these (uncomment necessary lib)


In [80]:
# !pip install geocoder
# !pip install geopy
# !pip install folium
# !pip install mysql

In [81]:
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np
import folium
from folium.plugins import FastMarkerCluster
import mysql.connector
import timeit

from functions.chronometer import check_time

***
# =-=-=-=-=-=-=-=-=-= SETAR DATA E RODOVIA =-=-=-=-=-=-=-=-=-=

In [82]:
data = '2020-01-07'
rodovia = 'dutra'

***
* Arquivo gerado a partir do KML do google earth

In [83]:
coord = pd.read_csv('../inputs/coordenadas/' + rodovia + '/' + rodovia + '_coord.csv')
coord

,Latitude,Longitude
0,-23.525513,-46.587819
1,-23.525083,-46.587481
2,-23.524845,-46.587295
3,-23.524453,-46.586983
4,-23.524057,-46.586675
...,...,...
9061,-22.816855,-43.323532
9062,-22.816929,-43.323396
9063,-22.817023,-43.323227
9064,-22.817088,-43.323105


In [84]:
addresses = pd.read_csv('../inputs/dados_rodovia/' + rodovia + '/' + rodovia + '_names.csv', delimiter=';')
addresses

,Rodovia,KM,State
0,Presidente Dutra,232,SP
1,Presidente Dutra,231,SP
2,Presidente Dutra,230,SP
3,Presidente Dutra,229,SP
4,Presidente Dutra,228,SP
...,...,...,...
398,Presidente Dutra,168,RJ
399,Presidente Dutra,167,RJ
400,Presidente Dutra,166,RJ
401,Presidente Dutra,165,RJ


***
# Divisor de kms

Gera as coordenadas que o arquivo KML gerou pela quantidade de KMs da rodovia. Caso o resultado da divisão não seja exata, é feito o "arredondamento" para que no final cada KM receba as coordenadas de forma correta.

In [85]:
km_total = len(addresses)                             # qual kms total da rodovia (inserir manualmente)
points_total = coord.shape[0]                         # total de pontos de coordenadas vindos do arquivo kml

if points_total % km_total == 0:                      # se a divião for exata
    div_fact = points_total // km_total               # fator de divisão
    print(div_fact)
else:
    div_fact_exato = km_total * (points_total // km_total)   # arredonda a divisor
    remove_n = points_total - div_fact_exato          # linhas a serem removidas
    drop_rows = np.random.choice(coord.index, remove_n, replace=False)  # linhas escolhidas aleatoriamente para serem removidas, a fim de arredondar a diferença
    coord = coord.drop(drop_rows)                     # remoção das linhas escolhidas
    coord = coord.reset_index(drop=True)              # reseta o index
    points_total_changed = coord.shape[0]             # pega o tamanho do dataframe
    div_fact = points_total_changed / km_total        # fator de divisão
    div_fact = int(div_fact)                          # transforma em inteiro


In [86]:
list_lat = pd.DataFrame(['%06.3f' % (coord.loc[i][0]) for i in range(0, len(coord), div_fact)], columns=['Latitude'])

In [87]:
list_lon = pd.DataFrame(['%06.3f' % (coord.loc[i][1]) for i in range(0, len(coord), div_fact)], columns=['Longitude'])

In [88]:
coord = pd.concat([list_lat, list_lon], axis=1)
coord

,Latitude,Longitude
0,-23.526,-46.588
1,-23.518,-46.581
2,-23.510,-46.574
3,-23.502,-46.565
4,-23.497,-46.558
...,...,...
398,-22.798,-43.359
399,-22.802,-43.351
400,-22.806,-43.343
401,-22.809,-43.338


***
# Map

Plota no mapa as coordenadas geradas

In [89]:
map1 = folium.Map(location=[-22.8923, -45.2887],
                  tiles='cartodbpositron',
                  zoom_start=9,
                  )

In [90]:
coord.apply(lambda row:folium.CircleMarker(location=[row["Latitude"], row["Longitude"]]).add_to(map1), axis=1)

0      <folium.vector_layers.CircleMarker object at 0...
1      <folium.vector_layers.CircleMarker object at 0...
2      <folium.vector_layers.CircleMarker object at 0...
3      <folium.vector_layers.CircleMarker object at 0...
4      <folium.vector_layers.CircleMarker object at 0...
                             ...                        
398    <folium.vector_layers.CircleMarker object at 0...
399    <folium.vector_layers.CircleMarker object at 0...
400    <folium.vector_layers.CircleMarker object at 0...
401    <folium.vector_layers.CircleMarker object at 0...
402    <folium.vector_layers.CircleMarker object at 0...
Length: 403, dtype: object

In [149]:
# map1

***

* Addresses

Para cada coordenada é feita a junção da descrição do KM correspondente

In [92]:
addresses['Latitude'] = ""
addresses['Longitude'] = ""

In [93]:
for x in range(len(addresses)):
    addresses.loc[x, 'Latitude'] = coord.loc[x, 'Latitude']
    addresses.loc[x, 'Longitude'] = coord.loc[x, 'Longitude']

# print(addresses.to_string())
addresses

,Rodovia,KM,State,Latitude,Longitude
0,Presidente Dutra,232,SP,-23.526,-46.588
1,Presidente Dutra,231,SP,-23.518,-46.581
2,Presidente Dutra,230,SP,-23.510,-46.574
3,Presidente Dutra,229,SP,-23.502,-46.565
4,Presidente Dutra,228,SP,-23.497,-46.558
...,...,...,...,...,...
398,Presidente Dutra,168,RJ,-22.798,-43.359
399,Presidente Dutra,167,RJ,-22.802,-43.351
400,Presidente Dutra,166,RJ,-22.806,-43.343
401,Presidente Dutra,165,RJ,-22.809,-43.338


***
* Save to CSV

In [94]:
addresses_to_csv = pd.DataFrame({
                                 'Rodovia'   : addresses['Rodovia'].astype(str),
                                 'KM'        : addresses['KM'],
                                 'State'     : addresses['State'].astype(str),
                                 'Latitude'  : addresses['Latitude'].astype(str),
                                 'Longitude' : addresses['Longitude'].astype(str),
                                })
addresses_to_csv.to_csv('../inputs/addresses/' + rodovia + '/addresses.csv', index=False)
addresses_to_csv

,Rodovia,KM,State,Latitude,Longitude
0,Presidente Dutra,232,SP,-23.526,-46.588
1,Presidente Dutra,231,SP,-23.518,-46.581
2,Presidente Dutra,230,SP,-23.510,-46.574
3,Presidente Dutra,229,SP,-23.502,-46.565
4,Presidente Dutra,228,SP,-23.497,-46.558
...,...,...,...,...,...
398,Presidente Dutra,168,RJ,-22.798,-43.359
399,Presidente Dutra,167,RJ,-22.802,-43.351
400,Presidente Dutra,166,RJ,-22.806,-43.343
401,Presidente Dutra,165,RJ,-22.809,-43.338


***
* Load CSV

In [95]:
addresses = pd.read_csv('../inputs/addresses/' + rodovia + '/addresses.csv', delimiter=',')
addresses

,Rodovia,KM,State,Latitude,Longitude
0,Presidente Dutra,232,SP,-23.526,-46.588
1,Presidente Dutra,231,SP,-23.518,-46.581
2,Presidente Dutra,230,SP,-23.510,-46.574
3,Presidente Dutra,229,SP,-23.502,-46.565
4,Presidente Dutra,228,SP,-23.497,-46.558
...,...,...,...,...,...
398,Presidente Dutra,168,RJ,-22.798,-43.359
399,Presidente Dutra,167,RJ,-22.802,-43.351
400,Presidente Dutra,166,RJ,-22.806,-43.343
401,Presidente Dutra,165,RJ,-22.809,-43.338


***
# Gerador de TABLE usando INSERT - ATIVAR MODULOS APENAS QUANDO FOR CRIAR TABLE NA BASE DE DADOS

* MySQL connector

* Delete Table

* Create Table

***

* Insert values into Table

* Criar Index

* Close Connections

***
# Query, extrair leituras por KM

* MySQL connector

In [57]:
connection = mysql.connector.MySQLConnection(user="root",
                                             password="",
                                             host='127.0.0.1',
                                             database='coldchain',
                                             port=3306)

* 7 dias

In [ ]:
start=timeit.default_timer()

df_rodovia_kms_7_days = pd.read_sql_query('''SELECT
                                                 k.id, k.km, k.state,
                                                 count(d.id) AS 7_days_nro_smartphones
                                             FROM datapoint AS d
                                                 JOIN %s_kms AS k
                                                 JOIN status
                                                     ON status.datapoint_id = d.id
                                             WHERE d.longitude > k.long_min AND d.longitude < k.long_max
                                                 AND d.latitude > k.lat_min AND d.latitude < k.lat_max
                                                 AND ((d.datetime > '2020-01-07' - interval 7 day) AND (d.datetime < '2020-01-07' - interval 6 day))
                                                 AND (d.geo_accuracy <= 700)
                                                 AND (d.session_id IS NULL)
                                                 AND (d.temperature IS NULL)
                                                 AND (status.bluetooth = 1)
                                             GROUP BY k.km
                                             ORDER BY k.id
                                             ;''' %rodovia , connection)

end=timeit.default_timer()

check_time(start, end)

df_rodovia_kms_7_days

In [ ]:
start=timeit.default_timer()

df_rodovia_kms_7_days_B = pd.read_sql_query('''SELECT
                                               k.id, k.km, k.state,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 00 AND 05 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 7_days_esp_00h_06hs,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 06 AND 11 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 7_days_esp_06_12hs,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 12 AND 17 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 7_days_esp_12_18,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 18 AND 23 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 7_days_esp_18_24hs,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 00 AND 23 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 7_days_total
                                             FROM datapoint AS d
                                               JOIN dutra_kms AS k
                                             WHERE d.longitude > k.long_min AND d.longitude < k.long_max
                                               AND d.latitude > k.lat_min AND d.latitude < k.lat_max
                                               AND ((d.datetime > '2020-01-07' - interval 7 day) AND (d.datetime < '2020-01-07' - interval 6 day))
                                               AND (d.geo_accuracy <= 700)
                                               AND (d.session_id IS NOT NULL)
                                               AND (d.temperature IS NULL)
                                             GROUP BY k.km
                                             ORDER BY k.id
                                           ;''', connection)

end=timeit.default_timer()

check_time(start, end)

df_rodovia_kms_7_days_B

* 6 dias

In [ ]:
start=timeit.default_timer()

df_rodovia_kms_6_days = pd.read_sql_query('''SELECT
                                              k.id, k.km, k.state,
                                              count(d.id) AS 6_days_nro_smartphones
                                           FROM datapoint AS d
                                              JOIN dutra_kms AS k
                                              JOIN status
                                                  ON status.datapoint_id = d.id
                                           WHERE d.longitude > k.long_min AND d.longitude < k.long_max
                                              AND d.latitude > k.lat_min AND d.latitude < k.lat_max
                                              AND ((d.datetime > '2020-01-07' - interval 6 day) AND (d.datetime < '2020-01-07' - interval 5 day))
                                              AND (d.geo_accuracy <= 700)
                                              AND (d.session_id IS NULL)
                                              AND (d.temperature IS NULL)
                                              AND (status.bluetooth = 1)
                                           GROUP BY k.km
                                           ORDER BY k.id
                                           ;''', connection)

end=timeit.default_timer()

check_time(start, end)

df_rodovia_kms_6_days

In [ ]:
start=timeit.default_timer()

df_rodovia_kms_6_days_B = pd.read_sql_query('''SELECT
                                               k.id, k.km, k.state,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 00 AND 05 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 6_days_esp_00h_06hs,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 06 AND 11 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 6_days_esp_06_12hs,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 12 AND 17 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 6_days_esp_12_18,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 18 AND 23 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 6_days_esp_18_24hs,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 00 AND 23 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 6_days_total
                                             FROM datapoint AS d
                                               JOIN dutra_kms AS k
                                             WHERE d.longitude > k.long_min AND d.longitude < k.long_max
                                               AND d.latitude > k.lat_min AND d.latitude < k.lat_max
                                               AND ((d.datetime > '2020-01-07' - interval 6 day) AND (d.datetime < '2020-01-07' - interval 5 day))
                                               AND (d.geo_accuracy <= 700)
                                               AND (d.session_id IS NOT NULL)
                                               AND (d.temperature IS NULL)
                                             GROUP BY k.km
                                             ORDER BY k.id
                                           ;''', connection)

end=timeit.default_timer()

check_time(start, end)

df_rodovia_kms_6_days_B

* 5 dias

In [ ]:
start=timeit.default_timer()

df_rodovia_kms_5_days = pd.read_sql_query('''SELECT
                                              k.id, k.km, k.state,
                                              count(d.id) AS 5_days_nro_smartphones
                                           FROM datapoint AS d
                                              JOIN dutra_kms AS k
                                              JOIN status
                                                  ON status.datapoint_id = d.id
                                           WHERE d.longitude > k.long_min AND d.longitude < k.long_max
                                              AND d.latitude > k.lat_min AND d.latitude < k.lat_max
                                              AND ((d.datetime > '2020-01-07' - interval 6 day) AND (d.datetime < '2020-01-07' - interval 5 day))
                                              AND (d.geo_accuracy <= 700)
                                              AND (d.session_id IS NULL)
                                              AND (d.temperature IS NULL)
                                              AND (status.bluetooth = 1)
                                           GROUP BY k.km
                                           ORDER BY k.id
                                           ;''', connection)

end=timeit.default_timer()

check_time(start, end)

df_rodovia_kms_5_days

In [ ]:
start=timeit.default_timer()

df_rodovia_kms_5_days_B = pd.read_sql_query('''SELECT
                                               k.id, k.km, k.state,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 00 AND 05 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 5_days_esp_00h_06hs,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 06 AND 11 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 5_days_esp_06_12hs,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 12 AND 17 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 5_days_esp_12_18,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 18 AND 23 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 5_days_esp_18_24hs,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 00 AND 23 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 5_days_total
                                             FROM datapoint AS d
                                               JOIN dutra_kms AS k
                                             WHERE d.longitude > k.long_min AND d.longitude < k.long_max
                                               AND d.latitude > k.lat_min AND d.latitude < k.lat_max
                                               AND ((d.datetime > '2020-01-07' - interval 5 day) AND (d.datetime < '2020-01-07' - interval 4 day))
                                               AND (d.geo_accuracy <= 700)
                                               AND (d.session_id IS NOT NULL)
                                               AND (d.temperature IS NULL)
                                             GROUP BY k.km
                                             ORDER BY k.id
                                           ;''', connection)

end=timeit.default_timer()

check_time(start, end)

df_rodovia_kms_5_days_B

* 4 dias

In [ ]:
start=timeit.default_timer()

df_rodovia_kms_4_days = pd.read_sql_query('''SELECT
                                              k.id, k.km, k.state,
                                              count(d.id) AS 4_days_nro_smartphones
                                           FROM datapoint AS d
                                              JOIN dutra_kms AS k
                                              JOIN status
                                                  ON status.datapoint_id = d.id
                                           WHERE d.longitude > k.long_min AND d.longitude < k.long_max
                                              AND d.latitude > k.lat_min AND d.latitude < k.lat_max
                                              AND ((d.datetime > '2020-01-07' - interval 4 day) AND (d.datetime < '2020-01-07' - interval 3 day))
                                              AND (d.geo_accuracy <= 700)
                                              AND (d.session_id IS NULL)
                                              AND (d.temperature IS NULL)
                                              AND (status.bluetooth = 1)
                                           GROUP BY k.km
                                           ORDER BY k.id
                                           ;''', connection)

end=timeit.default_timer()

check_time(start, end)

df_rodovia_kms_4_days

In [ ]:
start=timeit.default_timer()

df_rodovia_kms_4_days_B = pd.read_sql_query('''SELECT
                                               k.id, k.km, k.state,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 00 AND 05 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 4_days_esp_00h_06hs,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 06 AND 11 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 4_days_esp_06_12hs,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 12 AND 17 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 4_days_esp_12_18,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 18 AND 23 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 4_days_esp_18_24hs,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 00 AND 23 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 4_days_total
                                             FROM datapoint AS d
                                               JOIN dutra_kms AS k
                                             WHERE d.longitude > k.long_min AND d.longitude < k.long_max
                                               AND d.latitude > k.lat_min AND d.latitude < k.lat_max
                                               AND ((d.datetime > '2020-01-07' - interval 4 day) AND (d.datetime < '2020-01-07' - interval 3 day))
                                               AND (d.geo_accuracy <= 700)
                                               AND (d.session_id IS NOT NULL)
                                               AND (d.temperature IS NULL)
                                             GROUP BY k.km
                                             ORDER BY k.id
                                           ;''', connection)

end=timeit.default_timer()

check_time(start, end)

df_rodovia_kms_4_days_B

* 3 dias

In [ ]:
start=timeit.default_timer()

df_rodovia_kms_3_days = pd.read_sql_query('''SELECT
                                              k.id, k.km, k.state,
                                              count(d.id) AS 3_days_nro_smartphones
                                           FROM datapoint AS d
                                              JOIN dutra_kms AS k
                                              JOIN status
                                                  ON status.datapoint_id = d.id
                                           WHERE d.longitude > k.long_min AND d.longitude < k.long_max
                                              AND d.latitude > k.lat_min AND d.latitude < k.lat_max
                                              AND ((d.datetime > '2020-01-07' - interval 3 day) AND (d.datetime < '2020-01-07' - interval 2 day))
                                              AND (d.geo_accuracy <= 700)
                                              AND (d.session_id IS NULL)
                                              AND (d.temperature IS NULL)
                                              AND (status.bluetooth = 1)
                                           GROUP BY k.km
                                           ORDER BY k.id
                                           ;''', connection)

end=timeit.default_timer()

check_time(start, end)

df_rodovia_kms_3_days

In [ ]:
start=timeit.default_timer()

df_rodovia_kms_3_days_B = pd.read_sql_query('''SELECT
                                               k.id, k.km, k.state,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 00 AND 05 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 3_days_esp_00h_06hs,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 06 AND 11 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 3_days_esp_06_12hs,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 12 AND 17 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 3_days_esp_12_18,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 18 AND 23 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 3_days_esp_18_24hs,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 00 AND 23 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 3_days_total
                                             FROM datapoint AS d
                                               JOIN dutra_kms AS k
                                             WHERE d.longitude > k.long_min AND d.longitude < k.long_max
                                               AND d.latitude > k.lat_min AND d.latitude < k.lat_max
                                               AND ((d.datetime > '2020-01-07' - interval 3 day) AND (d.datetime < '2020-01-07' - interval 2 day))
                                               AND (d.geo_accuracy <= 700)
                                               AND (d.session_id IS NOT NULL)
                                               AND (d.temperature IS NULL)
                                             GROUP BY k.km
                                             ORDER BY k.id
                                           ;''', connection)

end=timeit.default_timer()

check_time(start, end)

df_rodovia_kms_3_days_B

* 2 dias

In [ ]:
start=timeit.default_timer()

df_rodovia_kms_2_days = pd.read_sql_query('''SELECT
                                              k.id, k.km, k.state,
                                              count(d.id) AS 2_days_nro_smartphones
                                           FROM datapoint AS d
                                              JOIN dutra_kms AS k
                                              JOIN status
                                                  ON status.datapoint_id = d.id
                                           WHERE d.longitude > k.long_min AND d.longitude < k.long_max
                                              AND d.latitude > k.lat_min AND d.latitude < k.lat_max
                                              AND ((d.datetime > '2020-01-07' - interval 2 day) AND (d.datetime < '2020-01-07' - interval 1 day))
                                              AND (d.geo_accuracy <= 700)
                                              AND (d.session_id IS NULL)
                                              AND (d.temperature IS NULL)
                                              AND (status.bluetooth = 1)
                                           GROUP BY k.km
                                           ORDER BY k.id
                                           ;''', connection)

end=timeit.default_timer()

check_time(start, end)

df_rodovia_kms_2_days

In [ ]:
start=timeit.default_timer()

df_rodovia_kms_2_days_B = pd.read_sql_query('''SELECT
                                               k.id, k.km, k.state,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 00 AND 05 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 2_days_esp_00h_06hs,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 06 AND 11 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 2_days_esp_06_12hs,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 12 AND 17 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 2_days_esp_12_18,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 18 AND 23 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 2_days_esp_18_24hs,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 00 AND 23 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 2_days_total
                                             FROM datapoint AS d
                                               JOIN dutra_kms AS k
                                             WHERE d.longitude > k.long_min AND d.longitude < k.long_max
                                               AND d.latitude > k.lat_min AND d.latitude < k.lat_max
                                               AND ((d.datetime > '2020-01-07' - interval 2 day) AND (d.datetime < '2020-01-07' - interval 1 day))
                                               AND (d.geo_accuracy <= 700)
                                               AND (d.session_id IS NOT NULL)
                                               AND (d.temperature IS NULL)
                                             GROUP BY k.km
                                             ORDER BY k.id
                                           ;''', connection)

end=timeit.default_timer()

check_time(start, end)

df_rodovia_kms_2_days_B

* 1 dia

In [ ]:
start=timeit.default_timer()

df_rodovia_kms_1_day = pd.read_sql_query('''SELECT
                                              k.id, k.km, k.state,
                                              count(d.id) AS 1_day_nro_smartphones
                                           FROM datapoint AS d
                                              JOIN dutra_kms AS k
                                              JOIN status
                                                  ON status.datapoint_id = d.id
                                           WHERE d.longitude > k.long_min AND d.longitude < k.long_max
                                              AND d.latitude > k.lat_min AND d.latitude < k.lat_max
                                              AND ((d.datetime > '2020-01-07' - interval 1 day) AND (d.datetime < '2020-01-07'))
                                              AND (d.geo_accuracy <= 700)
                                              AND (d.session_id IS NULL)
                                              AND (d.temperature IS NULL)
                                              AND (status.bluetooth = 1)
                                           GROUP BY k.km
                                           ORDER BY k.id
                                           ;''', connection)

end=timeit.default_timer()

check_time(start, end)

df_rodovia_kms_1_day

In [ ]:
start=timeit.default_timer()

df_rodovia_kms_1_day_B = pd.read_sql_query('''SELECT
                                               k.id, k.km, k.state,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 00 AND 05 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 1_day_esp_00h_06hs,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 06 AND 11 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 1_day_esp_06_12hs,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 12 AND 17 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 1_day_esp_12_18,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 18 AND 23 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 1_day_esp_18_24hs,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 00 AND 23 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS 1_day_total
                                             FROM datapoint AS d
                                               JOIN dutra_kms AS k
                                             WHERE d.longitude > k.long_min AND d.longitude < k.long_max
                                               AND d.latitude > k.lat_min AND d.latitude < k.lat_max
                                               AND ((d.datetime > '2020-01-07' - interval 1 day) AND (d.datetime < '2020-01-07'))
                                               AND (d.geo_accuracy <= 700)
                                               AND (d.session_id IS NOT NULL)
                                               AND (d.temperature IS NULL)
                                             GROUP BY k.km
                                             ORDER BY k.id
                                           ;''', connection)

end=timeit.default_timer()

check_time(start, end)

df_rodovia_kms_1_day_B

* dia atual, target

In [ ]:
start=timeit.default_timer()

df_rodovia_kms_day = pd.read_sql_query('''SELECT
                                              k.id, k.km, k.state,
                                              count(d.id) AS day_nro_smartphones
                                           FROM datapoint AS d
                                              JOIN dutra_kms AS k
                                              JOIN status
                                                  ON status.datapoint_id = d.id
                                           WHERE d.longitude > k.long_min AND d.longitude < k.long_max
                                              AND d.latitude > k.lat_min AND d.latitude < k.lat_max
                                              AND ((d.datetime > '2020-01-07') AND (d.datetime < '2020-01-07' + interval 1 day))
                                              AND (d.geo_accuracy <= 700)
                                              AND (d.session_id IS NULL)
                                              AND (d.temperature IS NULL)
                                              AND (status.bluetooth = 1)
                                           GROUP BY k.km
                                           ORDER BY k.id
                                           ;''', connection)

end=timeit.default_timer()

check_time(start, end)

df_rodovia_kms_day

In [ ]:
start=timeit.default_timer()

df_rodovia_kms_day_B = pd.read_sql_query('''SELECT
                                               k.id, k.km, k.state,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 00 AND 05 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS esp_00h_06hs,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 06 AND 11 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS esp_06_12hs,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 12 AND 17 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS esp_12_18,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 18 AND 23 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS esp_18_24hs,
                                               CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 00 AND 23 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS day_total
                                             FROM datapoint AS d
                                               JOIN dutra_kms AS k
                                             WHERE d.longitude > k.long_min AND d.longitude < k.long_max
                                               AND d.latitude > k.lat_min AND d.latitude < k.lat_max
                                               AND ((d.datetime > '2020-01-07') AND (d.datetime < '2020-01-07' + interval 1 day))
                                               AND (d.geo_accuracy <= 700)
                                               AND (d.session_id IS NOT NULL)
                                               AND (d.temperature IS NULL)
                                             GROUP BY k.km
                                             ORDER BY k.id
                                           ;''', connection)

end=timeit.default_timer()

check_time(start, end)

df_rodovia_kms_day_B

* Close Connections

In [ ]:
connection.close()

print('FINISH!')

***

* Export to CSV

In [ ]:
df_leituras_7_days_rodovia_kms = pd.DataFrame({
                                               'Id'                       : df_rodovia_kms_7_days['id'].astype(str),
                                               'state'                    : df_rodovia_kms_7_days['state'],
                                               'km'                       : df_rodovia_kms_7_days['km'].astype(str),
                                               '7_days_nro_smartphones'   : df_rodovia_kms_7_days['7_days_nro_smartphones'].astype(str),
                                              })
df_leituras_7_days_rodovia_kms.to_csv('../outputs_' + data + '/' + rodovia + '/leituras_7_days_' + rodovia + '.csv', index=False)
df_leituras_7_days_rodovia_kms

In [ ]:
df_leituras_7_days_rodovia_kms_B = pd.DataFrame({
                                                 'Id'                     : df_rodovia_kms_7_days_B['id'].astype(str),
                                                 'state'                  : df_rodovia_kms_7_days_B['state'],
                                                 'km'                     : df_rodovia_kms_7_days_B['km'].astype(str),
                                                 '7_days_esp_00h_06hs'    : df_rodovia_kms_7_days_B['7_days_esp_00h_06hs'].astype(str),
                                                 '7_days_esp_06_12hs'     : df_rodovia_kms_7_days_B['7_days_esp_06_12hs'].astype(str),
                                                 '7_days_esp_12_18'       : df_rodovia_kms_7_days_B['7_days_esp_12_18'].astype(str),
                                                 '7_days_esp_18_24hs'     : df_rodovia_kms_7_days_B['7_days_esp_18_24hs'].astype(str),
                                                 '7_days_total'           : df_rodovia_kms_7_days_B['7_days_total'].astype(str),
                                                })
df_leituras_7_days_rodovia_kms_B.to_csv('../outputs_' + data + '/' + rodovia + '/leituras_7_days_' + rodovia + '_B.csv', index=False)
df_leituras_7_days_rodovia_kms_B

In [ ]:
df_leituras_6_days_rodovia_kms = pd.DataFrame({
                                               'Id'                       : df_rodovia_kms_6_days['id'].astype(str),
                                               'state'                    : df_rodovia_kms_6_days['state'],
                                               'km'                       : df_rodovia_kms_6_days['km'].astype(str),
                                               '6_days_nro_smartphones'   : df_rodovia_kms_6_days['6_days_nro_smartphones'].astype(str),
                                              })
df_leituras_6_days_rodovia_kms.to_csv('../outputs_' + data + '/' + rodovia + '/leituras_6_days_' + rodovia + '.csv', index=False)
df_leituras_6_days_rodovia_kms

In [ ]:
df_leituras_6_days_rodovia_kms_B = pd.DataFrame({
                                                 'Id'                     : df_rodovia_kms_6_days_B['id'].astype(str),
                                                 'state'                  : df_rodovia_kms_6_days_B['state'],
                                                 'km'                     : df_rodovia_kms_6_days_B['km'].astype(str),
                                                 '6_days_esp_00h_06hs'    : df_rodovia_kms_6_days_B['6_days_esp_00h_06hs'].astype(str),
                                                 '6_days_esp_06_12hs'     : df_rodovia_kms_6_days_B['6_days_esp_06_12hs'].astype(str),
                                                 '6_days_esp_12_18'       : df_rodovia_kms_6_days_B['6_days_esp_12_18'].astype(str),
                                                 '6_days_esp_18_24hs'     : df_rodovia_kms_6_days_B['6_days_esp_18_24hs'].astype(str),
                                                 '6_days_total'           : df_rodovia_kms_6_days_B['6_days_total'].astype(str),
                                                })
df_leituras_6_days_rodovia_kms_B.to_csv('../outputs_' + data + '/' + rodovia + '/leituras_6_days_' + rodovia + '_B.csv', index=False)
df_leituras_6_days_rodovia_kms_B

In [ ]:
df_leituras_5_days_rodovia_kms = pd.DataFrame({
                                               'Id'                       : df_rodovia_kms_5_days['id'].astype(str),
                                               'state'                    : df_rodovia_kms_5_days['state'],
                                               'km'                       : df_rodovia_kms_5_days['km'].astype(str),
                                               '5_days_nro_smartphones'   : df_rodovia_kms_5_days['5_days_nro_smartphones'].astype(str),
                                              })
df_leituras_5_days_rodovia_kms.to_csv('../outputs_' + data + '/' + rodovia + '/leituras_5_days_' + rodovia + '.csv', index=False)
df_leituras_5_days_rodovia_kms

In [ ]:
df_leituras_5_days_rodovia_kms_B = pd.DataFrame({
                                                 'Id'                     : df_rodovia_kms_5_days_B['id'].astype(str),
                                                 'state'                  : df_rodovia_kms_5_days_B['state'],
                                                 'km'                     : df_rodovia_kms_5_days_B['km'].astype(str),
                                                 '5_days_esp_00h_06hs'    : df_rodovia_kms_5_days_B['5_days_esp_00h_06hs'].astype(str),
                                                 '5_days_esp_06_12hs'     : df_rodovia_kms_5_days_B['5_days_esp_06_12hs'].astype(str),
                                                 '5_days_esp_12_18'       : df_rodovia_kms_5_days_B['5_days_esp_12_18'].astype(str),
                                                 '5_days_esp_18_24hs'     : df_rodovia_kms_5_days_B['5_days_esp_18_24hs'].astype(str),
                                                 '5_days_total'           : df_rodovia_kms_5_days_B['5_days_total'].astype(str),
                                                })
df_leituras_5_days_rodovia_kms_B.to_csv('../outputs_' + data + '/' + rodovia + '/leituras_5_days_' + rodovia + '_B.csv', index=False)
df_leituras_5_days_rodovia_kms_B

In [ ]:
df_leituras_4_days_rodovia_kms = pd.DataFrame({
                                               'Id'                       : df_rodovia_kms_4_days['id'].astype(str),
                                               'state'                    : df_rodovia_kms_4_days['state'],
                                               'km'                       : df_rodovia_kms_4_days['km'].astype(str),
                                               '4_days_nro_smartphones'   : df_rodovia_kms_4_days['4_days_nro_smartphones'].astype(str),
                                              })
df_leituras_4_days_rodovia_kms.to_csv('../outputs_' + data + '/' + rodovia + '/leituras_4_days_' + rodovia + '.csv', index=False)
df_leituras_4_days_rodovia_kms

In [ ]:
df_leituras_4_days_rodovia_kms_B = pd.DataFrame({
                                                 'Id'                     : df_rodovia_kms_4_days_B['id'].astype(str),
                                                 'state'                  : df_rodovia_kms_4_days_B['state'],
                                                 'km'                     : df_rodovia_kms_4_days_B['km'].astype(str),
                                                 '4_days_esp_00h_06hs'    : df_rodovia_kms_4_days_B['4_days_esp_00h_06hs'].astype(str),
                                                 '4_days_esp_06_12hs'     : df_rodovia_kms_4_days_B['4_days_esp_06_12hs'].astype(str),
                                                 '4_days_esp_12_18'       : df_rodovia_kms_4_days_B['4_days_esp_12_18'].astype(str),
                                                 '4_days_esp_18_24hs'     : df_rodovia_kms_4_days_B['4_days_esp_18_24hs'].astype(str),
                                                 '4_days_total'           : df_rodovia_kms_4_days_B['4_days_total'].astype(str),
                                                })
df_leituras_4_days_rodovia_kms_B.to_csv('../outputs_' + data + '/' + rodovia + '/leituras_4_days_' + rodovia + '_B.csv', index=False)
df_leituras_4_days_rodovia_kms_B

In [ ]:
df_leituras_3_days_rodovia_kms = pd.DataFrame({
                                               'Id'                       : df_rodovia_kms_3_days['id'].astype(str),
                                               'state'                    : df_rodovia_kms_3_days['state'],
                                               'km'                       : df_rodovia_kms_3_days['km'].astype(str),
                                               '3_days_nro_smartphones'   : df_rodovia_kms_3_days['3_days_nro_smartphones'].astype(str),
                                              })
df_leituras_3_days_rodovia_kms.to_csv('../outputs_' + data + '/' + rodovia + '/leituras_3_days_' + rodovia + '.csv', index=False)
df_leituras_3_days_rodovia_kms

In [ ]:
df_leituras_3_days_rodovia_kms_B = pd.DataFrame({
                                                 'Id'                     : df_rodovia_kms_3_days_B['id'].astype(str),
                                                 'state'                  : df_rodovia_kms_3_days_B['state'],
                                                 'km'                     : df_rodovia_kms_3_days_B['km'].astype(str),
                                                 '3_days_esp_00h_06hs'    : df_rodovia_kms_3_days_B['3_days_esp_00h_06hs'].astype(str),
                                                 '3_days_esp_06_12hs'     : df_rodovia_kms_3_days_B['3_days_esp_06_12hs'].astype(str),
                                                 '3_days_esp_12_18'       : df_rodovia_kms_3_days_B['3_days_esp_12_18'].astype(str),
                                                 '3_days_esp_18_24hs'     : df_rodovia_kms_3_days_B['3_days_esp_18_24hs'].astype(str),
                                                 '3_days_total'           : df_rodovia_kms_3_days_B['3_days_total'].astype(str),
                                                })
df_leituras_3_days_rodovia_kms_B.to_csv('../outputs_' + data + '/' + rodovia + '/leituras_3_days_' + rodovia + '_B.csv', index=False)
df_leituras_3_days_rodovia_kms_B

In [ ]:
df_leituras_2_days_rodovia_kms = pd.DataFrame({
                                               'Id'                       : df_rodovia_kms_2_days['id'].astype(str),
                                               'state'                    : df_rodovia_kms_2_days['state'],
                                               'km'                       : df_rodovia_kms_2_days['km'].astype(str),
                                               '2_days_nro_smartphones'   : df_rodovia_kms_2_days['2_days_nro_smartphones'].astype(str),
                                              })
df_leituras_2_days_rodovia_kms.to_csv('../outputs_' + data + '/' + rodovia + '/leituras_2_days_' + rodovia + '.csv', index=False)
df_leituras_2_days_rodovia_kms

In [ ]:
df_leituras_2_days_rodovia_kms_B = pd.DataFrame({
                                                 'Id'                     : df_rodovia_kms_2_days_B['id'].astype(str),
                                                 'state'                  : df_rodovia_kms_2_days_B['state'],
                                                 'km'                     : df_rodovia_kms_2_days_B['km'].astype(str),
                                                 '2_days_esp_00h_06hs'    : df_rodovia_kms_2_days_B['2_days_esp_00h_06hs'].astype(str),
                                                 '2_days_esp_06_12hs'     : df_rodovia_kms_2_days_B['2_days_esp_06_12hs'].astype(str),
                                                 '2_days_esp_12_18'       : df_rodovia_kms_2_days_B['2_days_esp_12_18'].astype(str),
                                                 '2_days_esp_18_24hs'     : df_rodovia_kms_2_days_B['2_days_esp_18_24hs'].astype(str),
                                                 '2_days_total'           : df_rodovia_kms_2_days_B['2_days_total'].astype(str),
                                                })
df_leituras_2_days_rodovia_kms_B.to_csv('../outputs_' + data + '/' + rodovia + '/leituras_2_days_' + rodovia + '_B.csv', index=False)
df_leituras_2_days_rodovia_kms_B

In [ ]:
df_leituras_1_day_rodovia_kms = pd.DataFrame({
                                              'Id'                      : df_rodovia_kms_1_day['id'].astype(str),
                                              'state'                   : df_rodovia_kms_1_day['state'],
                                              'km'                      : df_rodovia_kms_1_day['km'].astype(str),
                                              '1_day_nro_smartphones'   : df_rodovia_kms_1_day['1_day_nro_smartphones'].astype(str),
                                             })
df_leituras_1_day_rodovia_kms.to_csv('../outputs_' + data + '/' + rodovia + '/leituras_1_day_' + rodovia + '.csv', index=False)
df_leituras_1_day_rodovia_kms

In [ ]:
df_leituras_1_day_rodovia_kms_B = pd.DataFrame({
                                                 'Id'                    : df_rodovia_kms_1_day_B['id'].astype(str),
                                                 'state'                 : df_rodovia_kms_1_day_B['state'],
                                                 'km'                    : df_rodovia_kms_1_day_B['km'].astype(str),
                                                 '1_day_esp_00h_06hs'    : df_rodovia_kms_1_day_B['1_day_esp_00h_06hs'].astype(str),
                                                 '1_day_esp_06_12hs'     : df_rodovia_kms_1_day_B['1_day_esp_06_12hs'].astype(str),
                                                 '1_day_esp_12_18'       : df_rodovia_kms_1_day_B['1_day_esp_12_18'].astype(str),
                                                 '1_day_esp_18_24hs'     : df_rodovia_kms_1_day_B['1_day_esp_18_24hs'].astype(str),
                                                 '1_day_total'           : df_rodovia_kms_1_day_B['1_day_total'].astype(str),
                                               })
df_leituras_1_day_rodovia_kms_B.to_csv('../outputs_' + data + '/' + rodovia + '/leituras_1_day_' + rodovia + '_B.csv', index=False)
df_leituras_1_day_rodovia_kms_B

In [ ]:
df_leituras_day_rodovia_kms = pd.DataFrame({
                                            'Id'                    : df_rodovia_kms_day['id'].astype(str),
                                            'state'                 : df_rodovia_kms_day['state'],
                                            'km'                    : df_rodovia_kms_day['km'].astype(str),
                                            'day_nro_smartphones'   : df_rodovia_kms_day['day_nro_smartphones'].astype(str),
                                           })
df_leituras_day_rodovia_kms.to_csv('../outputs_' + data + '/' + rodovia + '/leituras_day_' + rodovia + '.csv', index=False)
df_leituras_day_rodovia_kms

In [ ]:
df_leituras_day_rodovia_kms_B = pd.DataFrame({
                                              'Id'              : df_rodovia_kms_day_B['id'].astype(str),
                                              'state'           : df_rodovia_kms_day_B['state'],
                                              'km'              : df_rodovia_kms_day_B['km'].astype(str),
                                              'esp_00h_06hs'    : df_rodovia_kms_day_B['esp_00h_06hs'].astype(str),
                                              'esp_06_12hs'     : df_rodovia_kms_day_B['esp_06_12hs'].astype(str),
                                              'esp_12_18'       : df_rodovia_kms_day_B['esp_12_18'].astype(str),
                                              'esp_18_24hs'     : df_rodovia_kms_day_B['esp_18_24hs'].astype(str),
                                              'day_total'       : df_rodovia_kms_day_B['day_total'].astype(str),
                                             })
df_leituras_day_rodovia_kms_B.to_csv('../outputs_' + data + '/' + rodovia + '/leituras_day_' + rodovia + '_B.csv', index=False)
df_leituras_day_rodovia_kms_B